In [5]:
import csv
import ast
import os
import re

def remove_duplicates_from_restored(file_path):
    with open(file_path, mode='r+', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames
        rows = list(reader)
        
        file.seek(0)
        writer = csv.DictWriter(file, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        writer.writeheader()
        
        for row in rows:
            restored = ast.literal_eval(row['restored'])
            if len(restored) > 1 and isinstance(restored[1], list):
                # Deduplicate the nested list while preserving the order
                deduplicated_list = list(dict.fromkeys(restored[1]))
                # Update the 'restored' field with the deduplicated list
                row['restored'] = str([restored[0], deduplicated_list])
            writer.writerow(row)
        
        file.truncate()

# Iterate through all files in the directory
directory = 'results/LLM_enhanced/'
pattern = re.compile(r'llm_results_gpt4_0.8_doi_collectionSite_\d+_\d+(?:st|nd|rd|th)\.csv')

for filename in os.listdir(directory):
    if pattern.match(filename):
        file_path = os.path.join(directory, filename)
        remove_duplicates_from_restored(file_path)
